In [ ]:
import torch
from cholespy import CholeskySolverF, MatrixType
from utils import get_icosphere, get_coo_arrays
import igl
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import time
import numpy as np
import tqdm

import sksparse.cholmod as cholmod
import scipy.sparse as sp
import cupy as cp
import cupyx.scipy.sparse as cps
import cupyx.scipy.sparse.linalg as cpsl
from cupy import cusparse
from torch.utils.dlpack import to_dlpack
from torch.utils.dlpack import from_dlpack

matplotlib.rcParams['font.size'] = 18
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = r"""\usepackage{libertine}
\usepackage{amsmath}"""
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
sns.set(font_scale=1.1)

In [ ]:
def torch_to_cupy(x):
    return cp.from_dlpack(to_dlpack(x))

In [ ]:
def cupy_to_torch(x):
    return from_dlpack(x.toDlpack())

In [ ]:
def prepare(A, transpose, blocking=True, level_info=True):
    import cupy as _cupy
    from cupy.cuda import device as _device
    from cupy_backends.cuda.libs import cusparse as _cusparse
    import cupyx.scipy.sparse

    policy = _cusparse.CUSPARSE_SOLVE_POLICY_USE_LEVEL if level_info \
        else _cusparse.CUSPARSE_SOLVE_POLICY_NO_LEVEL
    algo = 1 if blocking else 0

    transa = _cusparse.CUSPARSE_OPERATION_TRANSPOSE if transpose \
        else _cusparse.CUSPARSE_OPERATION_NON_TRANSPOSE
    transb = _cusparse.CUSPARSE_OPERATION_TRANSPOSE
    fill_mode = _cusparse.CUSPARSE_FILL_MODE_LOWER

    if cupyx.scipy.sparse.isspmatrix_csc(A):
        A = A.T
        transa = 1 - transa
        fill_mode = 1 - fill_mode

    assert cupyx.scipy.sparse.isspmatrix_csr(A)

    handle = _device.get_cusparse_handle()
    info = _cusparse.createCsrsm2Info()
    m = A.shape[0]
    alpha = np.array(1, dtype=np.float32)
    desc = _cusparse.createMatDescr()
    _cusparse.setMatType(desc, _cusparse.CUSPARSE_MATRIX_TYPE_GENERAL)
    _cusparse.setMatIndexBase(desc, _cusparse.CUSPARSE_INDEX_BASE_ZERO)
    _cusparse.setMatFillMode(desc, fill_mode)
    _cusparse.setMatDiagType(desc, _cusparse.CUSPARSE_DIAG_TYPE_NON_UNIT)

    nrhs = 128
    ldb = nrhs

    ws_size = _cusparse.scsrsm2_bufferSizeExt(
        handle, algo, transa, transb, m, nrhs, A.nnz, alpha.ctypes.data,
        desc, A.data.data.ptr, A.indptr.data.ptr, A.indices.data.ptr,
        0, ldb, info, policy)

    ws = _cupy.empty((ws_size,), dtype=np.int8)

    _cusparse.scsrsm2_analysis(
         handle, algo, transa, transb, m, nrhs, A.nnz, alpha.ctypes.data,
         desc, A.data.data.ptr, A.indptr.data.ptr,
         A.indices.data.ptr, 0, ldb, info, policy, ws.data.ptr)

    def solver(b):
        _cusparse.scsrsm2_solve(
            handle, algo, transa, transb, m, nrhs, A.nnz, alpha.ctypes.data,
            desc, A.data.data.ptr, A.indptr.data.ptr, A.indices.data.ptr,
            b.data.ptr, ldb, info, policy, ws.data.ptr)

    return solver

In [ ]:
class Cholesky:
    """
    Cholesky decomposition of a matrix
    """
    def __init__(self, n_verts, M_cpu):
        """
        M: pytorch sparse tensor, the matrix to decompose. Assumed to be spd
        """

        factor = cholmod.cholesky(M_cpu, ordering_method='nesdis', mode='simplicial')
        L, P = factor.L(), factor.P()
        # Invert the permutation
        Pi = np.argsort(P).astype(np.int32)

        # Transfer to GPU as cupy arrays
        self.L = cps.csc_matrix(L.astype(np.float32))
        self.U = self.L.T
        self.P = cp.array(P)
        self.Pi = cp.array(Pi)
        
        self.solver_1 = prepare(self.L, False, False, True)
        self.solver_2 = prepare(self.L, True, False, True)

    def solve(self, b):
        tmp = torch_to_cupy(b)[self.P]
        self.solver_1(tmp)
        self.solver_2(tmp)
        sol = cupy_to_torch(tmp[self.Pi])
        
        return sol

In [ ]:
lambda_ = 2.0
n_verts, f = get_icosphere(6)
n_faces = len(f)
n_runs = np.array([20, 100])
n_rhs = 128
timings_cupy = np.zeros(2, dtype=float)
timings_ours_cuda = np.zeros(2, dtype=float)

## Baseline GPU timing

In [ ]:
for i in tqdm.trange(n_runs[0]):
    # change the matrix, but don't time this part
    values, idx = get_coo_arrays(n_verts, f, i+1)
    M_cpu = sp.coo_matrix((values, idx))
    
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
       
    start.record()
    
    solver_cupy = Cholesky(n_verts, M_cpu) 
    
    end.record()
    torch.cuda.synchronize()
    
    timings_cupy[0] += start.elapsed_time(end)

for i in tqdm.trange(n_runs[1]):
    
    b_numpy = np.random.uniform(size=(n_verts, n_rhs))
    b_torch = torch.tensor(b_numpy, device='cuda', dtype=torch.float32)
    
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
       
    start.record()
    
    x_cusparse = solver_cupy.solve(b_torch)
    
    end.record()
    torch.cuda.synchronize()
    
    timings_cupy[1] += start.elapsed_time(end)

timings_cupy /= n_runs

## Our GPU timing

In [ ]:
for i in tqdm.trange(n_runs[0]):
    
    values, idx = get_coo_arrays(n_verts, f, i+1)
    ii = torch.tensor(idx[0], device='cuda')
    jj = torch.tensor(idx[1], device='cuda')
    x = torch.tensor(values, device='cuda')
    
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
       
    start.record()
    solver_ours = CholeskySolverF(n_verts, ii, jj, x, MatrixType.COO)
    
    end.record()
    torch.cuda.synchronize()
    
    timings_ours_cuda[0] += start.elapsed_time(end)

for i in tqdm.trange(n_runs[1]):
    
    b_numpy = np.random.uniform(size=(n_verts, n_rhs))
    b_torch = torch.tensor(b_numpy, device='cuda', dtype=torch.float32)
    
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
       
    start.record()
    
    x_torch = torch.zeros_like(b_torch)
    solver_ours.solve(b_torch, x_torch)
    
    end.record()
    torch.cuda.synchronize()
    
    timings_ours_cuda[1] += start.elapsed_time(end)


timings_ours_cuda /= n_runs

## Baseline CPU timing

In [ ]:
timings_sksparse = np.zeros(2, dtype=float)
timings_ours_cpu = np.zeros(2, dtype=float)

In [ ]:
for i in tqdm.trange(n_runs[0]):
    # change the matrix, but don't time this part
    values, idx = get_coo_arrays(n_verts, f, i+1)
    M_cpu = sp.coo_matrix((values, idx))
    
    start = time.perf_counter()
    
    solver_sksparse = cholmod.cholesky(M_cpu, ordering_method='nesdis', mode='simplicial')

    end = time.perf_counter()
    
    timings_sksparse[0] += end - start

for i in tqdm.trange(n_runs[1]):
    
    b_numpy = np.random.uniform(size=(n_verts, n_rhs))
    
    start = time.perf_counter()
    
    x_sksparse = solver_sksparse(b_numpy)
    
    end = time.perf_counter()
    
    timings_sksparse[1] += end - start

timings_sksparse /= n_runs / 1000

## Our CPU timing

In [ ]:
for i in tqdm.trange(n_runs[0]):
    
    values, idx = get_coo_arrays(n_verts, f, i+1)
    
    start = time.perf_counter()
        
    solver_ours = CholeskySolverF(n_verts, idx[0], idx[1], values, MatrixType.COO)
    
    end = time.perf_counter()
    
    timings_ours_cpu[0] += end - start

for i in tqdm.trange(n_runs[1]):
    
    b_numpy = np.random.uniform(size=(n_verts, n_rhs)).astype(np.float32)
    x_numpy = np.zeros_like(b_numpy)
    
    start = time.perf_counter()
    
    solver_ours.solve(b_numpy, x_numpy)
    
    end = time.perf_counter()
    
    timings_ours_cpu[1] += end - start


timings_ours_cpu /= n_runs / 1000

# Figure

In [ ]:
titles = ["CUDA Version", "CPU Version"]
steps = ['Analysis', 'Solve']
labels = [["CuSparse", "Ours (CUDA)"], ["Scikit-Sparse", "Ours (CPU)"]]
timings_ours = [timings_ours_cuda, timings_ours_cpu]
timings_ref = [timings_cupy, timings_sksparse]

x = np.arange(2)  # the label locations
width = 0.35  # the width of the bars

aspect = 4/3
nrows = 1
ncols = 2

base_size = 10
w = ncols * base_size
h = nrows * base_size / aspect

fontsize = 24

fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(w,h))

for i, (names, t_ref, t_ours) in enumerate(zip(labels, timings_ref, timings_ours)):
    rects1 = ax[i].bar(x - width/2, t_ref, width, label=names[0])
    rects2 = ax[i].bar(x + width/2, t_ours, width, label=names[1])
    
    if i == 0:
        ax[i].set_ylabel('Elapsed time (ms)', fontsize=fontsize)
    
    ax[i].set_xticks(x, steps, fontsize=18)
    ax[i].legend()

    bar_label_1 = [f"{time:.2f}" for time in t_ref]
    bar_label_2 = [f"{t_ours[i]:.2f} (x{t_ours[i]/t_ref[i]:.2f})" for i in range(len(t_ref))]

    ax[i].bar_label(rects1, labels=bar_label_1, fontsize=15)
    ax[i].bar_label(rects2, labels=bar_label_2, fontsize=15)
    
    ax[i].set_title(titles[i], y=-0.2, fontsize=fontsize)
    ax[i].set_ylim(5, 400)

    plt.setp(ax[i], yscale='log')

plt.savefig("benchmark.jpg", format='jpg', dpi=500, bbox_inches='tight', pad_inches=0.1)